# Monitoreo de Cambios Agrícolas con Sentinel-1 SAR en los Llanos Orientales de Colombia

**Análisis Integrado de Detección de Cambios en Municipios de Casanare y Meta**

---

## Resumen Ejecutivo

Este notebook implementa un sistema completo de detección de cambios usando imágenes de radar Sentinel-1 SAR (Synthetic Aperture Radar) en Google Earth Engine para identificar cambios potencialmente asociados a actividad agrícola en 9 municipios seleccionados de los departamentos de Casanare y Meta, Colombia.

### Contexto Regional

Los Llanos Orientales de Colombia son una región agrícola de gran importancia, con extensos cultivos de **arroz**, **soya** y **palma de aceite** \[1\]. La constante presencia de nubes durante gran parte del año impide que los satélites ópticos tradicionales tomen imágenes claras de la superficie \[2\]. El radar Sentinel-1 supera este obstáculo al no depender de la luz solar, pudiendo atravesar las nubes y obtener imágenes consistentes cada 6-12 días \[3\].

### Objetivos

1. Implementar preprocesamiento consistente de imágenes Sentinel-1 GRD
2. Aplicar múltiples métodos de detección de cambios (diferencias temporales, anomalías, NDCV)
3. Clasificar y cuantificar cambios por municipio
4. Interpretar cambios en contexto agrícola regional

### Metodología

El análisis se basa en la metodología de análisis estadístico de cambios en series temporales de Sentinel-1 propuesta por Canty et al. \[4\] y el test de Wishart complejo de Conradsen et al. \[5\].

### Área de Estudio

**Departamento del Meta:**
- Puerto López (código DANE: 50-573)
- Castilla La Nueva (50-150)
- San Carlos de Guaroa (50-680)
- Cabuyaro (50-124)

**Departamento de Casanare:**
- Tauramena (85-410)
- Yopal (85-001)
- Aguazul (85-010)
- Nunchía (85-225)
- Villanueva (85-440)

---

## Referencias Principales

\[1\] MADR, "Evaluación Agropecuaria del Departamento del Meta y Casanare," Ministerio de Agricultura y Desarrollo Rural de Colombia, 2023.

\[2\] J. Reiche et al., "Combining satellite data for better tropical forest monitoring," *Nat. Clim. Change*, vol. 6, pp. 120–122, 2016. https://doi.org/10.1038/nclimate2919

\[3\] ESA, "Sentinel-1 User Handbook," European Space Agency, Tech. Rep. ESA-EOEP-CSCOP-TN-13-0001, 2013.

\[4\] M. J. Canty, A. A. Nielsen, H. Skriver, and K. Conradsen, "Statistical analysis of changes in Sentinel-1 time series on the Google Earth Engine," *Remote Sens.*, vol. 12, no. 1, p. 46, 2020. https://doi.org/10.3390/rs12010046

\[5\] K. Conradsen, A. A. Nielsen, J. Schou, and H. Skriver, "A test statistic in the complex Wishart distribution and its application to change detection in polarimetric SAR data," *IEEE Trans. Geosci. Remote Sens.*, vol. 41, no. 1, pp. 4–19, 2003. https://doi.org/10.1109/TGRS.2002.808066

*Ver archivo `references.bib` para bibliografía completa en formato BibTeX.*

## 1. Configuración del Entorno y Carga de Librerías

### 1.1. Instalación de Dependencias

Para ejecutar este notebook, primero instale las dependencias necesarias:

```bash
pip install earthengine-api geemap geopandas fiona matplotlib seaborn pandas numpy plotly jupyter
```

### 1.2. Autenticación de Google Earth Engine

**Importante**: Antes de ejecutar el notebook por primera vez, autentíquese con Google Earth Engine:

```python
import ee
ee.Authenticate()
```

In [ ]:
# Importar librerías
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.1)
sns.set_palette("Set2")

# Crear directorio de datos si no existe
Path("data").mkdir(exist_ok=True)

print("✓ Librerías importadas exitosamente")
print(f"✓ Versión de geemap: {geemap.__version__}")

In [ ]:
# Inicializar Google Earth Engine
try:
    ee.Initialize()
    print("✓ Earth Engine inicializado correctamente")
except Exception as e:
    print("⚠ Error al inicializar Earth Engine")
    print("Por favor ejecute: ee.Authenticate()")
    print(f"Error: {e}")